# Automated ML

Importing all needed dependencies to complete the project.

In [4]:
import logging
import os
import json
import csv
import numpy as np
import pandas as pd
import pkg_resources
import joblib

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core import Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from sklearn.preprocessing import StandardScaler

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.26.0


## Dataset

### Overview
This dataset comes from the Diabetes and Digestive and Kidney Disease National Institutes. The purpose of this dataset is to diagnose whether or not a patient is diabetic, on the basis of certain diagnostic measures in the dataset. The selection of these instances from a larger database was subject to several restrictions. All patients are women from the Indian heritage of Pima, at least 21 years old. 
https://www.kaggle.com/uciml/pima-indians-diabetes-database

Task
Predict the "Outcome" column based on the input features, either the patient has diabetes or not.

Reference: http://www.arogyaworld.org/wp-content/uploads/2010/10/ArogyaWorld_IndiaDiabetes_FactSheets_CGI2013_web.pdf

The dataset has nine features as follow:
- Pregnancies: Number pregnancy times (int).
- Glucose: Plasma glucose concentration level (int).
- BloodPressure: Diastolic blood pressure level in mm Hg(int).
- SkinThickness: skinfold thickness in mm(int).
- Insulin: two-hour serum insulin measured by mu U/ml(int).
- BMI: Body mass index (float).
- DiabetesPedigreeFunction: Diabetes pedigree function(float).
- Age: age in years 21 and above(int).
- Outcome: Target column 0 or 1, 0 = Not diabetes, 1 = diabetes(int).

In [5]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl'

experiment=Experiment(ws, experiment_name)

In [6]:
# Load the registered dataset from workspace
dataset = Dataset.get_by_name(ws, name='Diabetes')

# Convert the dataset to dataframe
df = dataset.to_pandas_dataframe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 769 entries, 0 to 768
Data columns (total 9 columns):
Column1    769 non-null object
Column2    769 non-null object
Column3    769 non-null object
Column4    769 non-null object
Column5    769 non-null object
Column6    769 non-null object
Column7    769 non-null object
Column8    769 non-null object
Column9    769 non-null object
dtypes: object(9)
memory usage: 54.2+ KB


In [7]:
df.describe()

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
count,769,769,769,769,769,769,769,769,769
unique,18,137,48,52,187,249,518,53,3
top,1,100,70,0,0,32,0.258,22,0
freq,135,17,57,227,374,13,6,72,500


In [8]:
#Display the first five records of the dataset
df.head()

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
1,6,148,72,35,0,33.6,0.627,50,1
2,1,85,66,29,0,26.6,0.351,31,0
3,8,183,64,0,0,23.3,0.672,32,1
4,1,89,66,23,94,28.1,0.167,21,0


In [9]:
# Create CPU cluster
amlcompute_cluster_name = "notebook138769"

# Verify if cluster does not exist otherwise use the existing one
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_V2',
                                                           vm_priority = 'lowpriority', 
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

Overview of the automl settings and configuration used for this experiment:

- "experiment_timeout_minutes": set to 30 minutes. The experiment will timeout after that period to avoid wasting resources.
- "max_concurrent_iterations": is set to 4. The max number of concurrent iterations to be run in parallel at the same time.
- "primary_metric" : is set to 'accuracy', which is a sutible metric for classification problems.
- "n_cross_validations": is set to 5, therefore the training and validation sets will be divided into five equal sets.
- "iterations": the number of iterations for the experiment is set to 20. It's a reasonable number and would provide the intendable result for the given dataset.
- compute_target: set to the project cluster to run the experiment.
- task: set to 'classification' since our target to predict whether the patient has diabetes or not.
- training_data: the loaded dataset for the project.
- label_column_name: set to the result/target colunm in the dataset 'Outcome' (0 or 1).
- enable_early_stopping: is enabled to terminate the experiment if the accuracy score is not showing improvement over time.
- featurization = is set to 'auto', it's an indicator of whether implementing a featurization step to preprocess/clean the dataset automatically or not. In our case, the preprocessing was applied for the numerical columns which normally involve treating missing values, cluster distance, the weight of evidence...etc.
- debug_log: errors will be logged into 'automl_errors.log'.


In [10]:
# Automl settings 
from azureml.train.automl import AutoMLConfig

automl_settings = {"experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 3,
    "iterations": 24
    
}

# Automl config 
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = 'classification',
                             training_data=dataset,
                             label_column_name='Column9',
                             enable_early_stopping= True,
                             featurization = 'auto',
                             debug_log = 'automl_errors.log',
                             **automl_settings
                            )




In [13]:
# Submit experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
Running on remote compute: notebook138769


Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_aab911f6-6cfc-47ff-a2a7-e97204230e51,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|1                    

## Run Details

The best model has resulted from the AutoML experiment from VotingEnsemble model. The Voting Ensemble model takes a majority vote of several algorithms which makes it surpass individual algorithms and minimize the bias. 

Use the `RunDetails` widget to show the different experiments.

In [14]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [15]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_aab911f6-6cfc-47ff-a2a7-e97204230e51,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|1                                |Outcome                          |769                                   |
+---------------------------------+----------

{'runId': 'AutoML_aab911f6-6cfc-47ff-a2a7-e97204230e51',
 'target': 'notebook138769',
 'status': 'Completed',
 'startTimeUtc': '2021-04-17T06:25:13.312583Z',
 'endTimeUtc': '2021-04-17T06:45:35.747417Z',
 'properties': {'num_iterations': '24',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'notebook138769',
  'AMLSettingsJsonString': '{"path":null,"name":"automl","subscription_id":"6b4af8be-9931-443e-90f6-c4c34a1f9737","resource_group":"aml-quickstarts-142899","workspace_name":"quick-starts-ws-142899","region":"southcentralus","compute_target":"notebook138769","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":24,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":0.0,"validation_size":0.0,"

In [19]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_aab911f6-6cfc-47ff-a2a7-e97204230e51,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Best Model

The cell below shows the best model from the automl experiments and display all the properties of the model.



In [27]:
# Retrieve and save best automl model
best_model, model = remote_run.get_output()
print(best_model)
print(model.steps)

Run(Experiment: automl,
Id: AutoML_aab911f6-6cfc-47ff-a2a7-e97204230e51_22,
Type: azureml.scriptrun,
Status: Completed)


AttributeError: 'NoneType' object has no attribute 'steps'

In [28]:
print(model)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: automl,
Id: AutoML_0c1155d7-c4c9-4662-abe0-5a612296c3c0_22,
Type: azureml.scriptrun,
Status: Completed)
[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                feature_sweeping_config=None, feature_sweeping_timeout=None,
                featurization_config=None, force_text_dnn=None,
                is_cross_validation=None, is_onnx_compatible=None, logger=None,
                observer=None, task=None, working_dir=None)), ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('19',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('logisticregression',
                                                            Log

In [25]:
print(best_model)

Run(Experiment: automl,
Id: AutoML_aab911f6-6cfc-47ff-a2a7-e97204230e51_22,
Type: azureml.scriptrun,
Status: Completed)


In [26]:
remote_run.get_metrics()

{'experiment_status': ['DatasetEvaluation',
  'FeaturesGeneration',
  'DatasetFeaturization',
  'DatasetFeaturizationCompleted',
  'DatasetBalancing',
  'DatasetCrossValidationSplit',
  'ModelSelection',
  'BestRunExplainModel',
  'ModelExplanationDataSetSetup',
  'PickSurrogateModel',
  'EngineeredFeatureExplanations',
  'EngineeredFeatureExplanations',
  'RawFeaturesExplanations',
  'RawFeaturesExplanations',
  'BestRunExplainModel'],
 'experiment_status_description': ['Gathering dataset statistics.',
  'Generating features for the dataset.',
  'Beginning to fit featurizers and featurize the dataset.',
  'Completed fit featurizers and featurizing the dataset.',
  'Performing class balancing sweeping',
  'Generating individually featurized CV splits.',
  'Beginning model selection.',
  'Best run model explanations started',
  'Model explanations data setup completed',
  'Choosing LinearModel as the surrogate model for explanations',
  'Computation of engineered features started',
  'C

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In the cell below, we have registered the model, created an inference config and deployed the model as a web service.

In [29]:
# Registring the best model
model = best_model.register_model(model_name='automl-best-model',model_path='outputs/model.pkl')


In [30]:
# Get automl environment with its dependencies
environment = Environment.get(ws, "AzureML-AutoML")

In [31]:
inference_config = InferenceConfig(entry_script='scoring.py',
                                   environment=environment)
service_name = 'automl-deploy'
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,
                       overwrite=True
                      )
service.wait_for_deployment(show_output=True)

scoring_uri = service.scoring_uri
print(scoring_uri)

ERROR:azureml._model_management._util:entry_script scoring.py doesn't exist. entry_script should be path relative to current working directory



WebserviceException: WebserviceException:
	Message: entry_script scoring.py doesn't exist. entry_script should be path relative to current working directory
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "entry_script scoring.py doesn't exist. entry_script should be path relative to current working directory"
    }
}

In [ ]:
# Enable app insights
service.update(enable_app_insights=True)

In the cell below, we have sent a request to the web service to test it.

In [ ]:
import requests
import json

#two set of data to score, so we get two results back
data = {"data": [{"Pregnancies": 5, 
     "Glucose": 150, 
     "BloodPressure": 70, 
     "SkinThickness": 40, 
     "Insulin": 10, 
     "BMI": 36.5, 
     "DiabetesPedigreeFunction": 0.627, 
     "Age": 30},

    {"Pregnancies": 5, 
     "Glucose": 90, 
     "BloodPressure": 70, 
     "SkinThickness": 34, 
     "Insulin": 20, 
     "BMI": 26.5, 
     "DiabetesPedigreeFunction": 0.351, 
     "Age": 28},
      ]}
    
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())
print("Case 0: Not Diabetes, Case 1: Diabetes.")

In [ ]:
data = [{"Pregnancies": 5, 
     "Glucose": 150, 
     "BloodPressure": 70, 
     "SkinThickness": 40, 
     "Insulin": 10, 
     "BMI": 36.5, 
     "DiabetesPedigreeFunction": 0.627, 
     "Age": 30},

    {"Pregnancies": 5, 
     "Glucose": 90, 
     "BloodPressure": 70, 
     "SkinThickness": 34, 
     "Insulin": 20, 
     "BMI": 26.5, 
     "DiabetesPedigreeFunction": 0.351, 
     "Age": 28},
      ]

print(data)

In [ ]:
# test using service instance
input_data = json.dumps({
    'data': data
})

output = service.run(input_data)
output

Print the logs of the web service and delete the service

In [ ]:
logs = service.get_logs()
logs

In [ ]:
service.delete()